# Electrical Power Generation 2

## Objective and Prerequisites

This example will teach you how to choose an optimal set of power stations to turn on in order to satisfy anticipated power demand over a 24-hour time horizon – but gives you the option of using hydroelectric power plants to satisfy that demand.

This model is example 16 from the fifth edition of Model Building in Mathematical Programming by H. Paul Williams on pages 271-272 and 326-327.

This example is at the intermediate level, where we assume that you know Python and the Gurobi Python API and that you have some knowledge of building mathematical optimization models.

This project was done in the context of the PSL Week with Ms Demassey.

---

## Problem Description

In this problem, thermal power generation units are grouped into three distinct types, with different characteristics for each type (power output, cost per megawatt hour, startup cost, etc.).  A unit can be on or off, with a startup cost associated with transitioning from off to on, and power output that can fall anywhere between a specified minimum and maximum value when the unit is on.  There are also two hydroelectric plants available, also with different cost and power generation characteristics.  A 24-hour time horizon is divided into 5 discrete time periods, each with an expected total power demand.  The model decides which units to turn on, and when, in order to satisfy demand for each time period.  The model also captures a reserve requirement, where the selected power plants must be capable of increasing their output, while still respecting their maximum output, in order to cope with the situation where actual demand exceeds predicted demand.
In this problem, the differentiation of the thermal plant is considered.

A set of generators is available to satisfy power demand for the following day.  Anticipated demand is as follows:

| Time Period | Demand (megawatts) |
| --- | --- |
| 12 pm to 6 am | 15000 |
| 6 am to 9 am | 30000 |
| 9 am to 3 pm | 25000 |
| 3 pm to 6 pm | 40000 |
| 6 pm to 12 pm | 27000 |

Thermal generators are grouped into three types, with the following minimum and maximum output for each type (when they are on):

| Type | Number available | Minimum output (MW) | Maximum output (MW) |
| --- | --- | --- | --- |
| 0 | 12 |  850 | 2000 |
| 1 | 10 | 1250 | 1750 |
| 2 | 5 | 1500 | 4000 |

There are costs associated with using a thermal generator: a cost per hour when the generator is on (and generating its minimum output), a cost per megawatt hour above its minimum, and a startup cost for turning a generator on:

| Type | Cost per hour (when on) | Cost per MWh above minimum | Startup cost |
| --- | --- | --- | --- |
| 0 | $\$1000$ | $\$2.00$ | $\$2000$ |
| 1 | $\$2600$ | $\$1.30$ | $\$1000$ |
| 2 | $\$3000$ | $\$3.00$ | $\$500$ |

Two hydroelectric generators are also available, each with a fixed power output (when on):

| Hydro plant | Output (MW) |
| --- | --- |
| A | 900 |
| B | 1400 |

The costs associated with using a hydro plant are slightly different.  There's an hourly cost, but it is much smaller than the hourly cost of a thermal generator.  The real cost for a hydroelectric plant comes from depletion of the water in the reservoir, which happens at different rates for the two units.  The reservoir must be replenished before the end of the time horizon by pumping water into it, which consumes electricity.  A hydroelectric plant also has a startup cost.

| Hydro plant | Cost per hour (when on) | Startup cost | Reservoir depth reduction (m/hr) |
| --- | --- | --- | --- |
| A | $\$90$ | $\$1500$ | 0.31 |
| B | $\$150$ | $\$1200$ | 0.47 |

Pumping water into the reservoir consumes electricity at a rate of 3000 MWh of electricity per meter of height.  The height of the reservoir at the end of the time horizon must be equal to the height at the beginning.

Generators must meet predicted demand, but they must also have sufficient reserve capacity to be able to cope with the situation where actual demand exceeds predicted demand.  For this model, the set of selected thermal generators plus the set of hydro generators must be able to produce as much as 115% of predicted demand.

Which generators should be committed to meet anticipated demand in order to minimize total cost?

---

## Model formulation

### Sets and Indices

$t \in \text{Types}=\{0,1,2\}$: Types of thermal generators.

$h \in \text{HydroUnits}=\{0,1\}$: Two hydro generators.

$p \in \text{Periods}=\{0,1,2,3,4\}$: Time periods.

Added:

$i \in \text{ThermalUnits}_{t} = \{0,1,2,3,4,5,6,7,8,9,10,11\} \ \text{if t=0},
\{0,1,2,3,4,5,6,7,8,9\} \ \text{if t=1},
\{0,1,2,3,4\} \ \text{if t=2}$: Units of thermal generators

### Parameters

$\text{period_hours}_{p} \in \mathbb{N}^+$: Number of hours in each time period.

$\text{demand}_p \in \mathbb{R}^+$: Total power demand for time period $p$.

$\text{generators}_t \in \mathbb{N}^+$: Number of thermal generators of type $t$.

$\text{start0} \in \mathbb{N}^+$: Number of thermal generators that are on at the beginning of the time horizon (and available in time period 0 without paying a startup cost).

$\text{min_load}_t \in \mathbb{R}^+$: Minimum output for thermal generator type $t$ (when on).

$\text{max_load}_t \in \mathbb{R}^+$: Maximum output for thermal generator type $t$.

$\text{base_cost}_t \in \mathbb{R}^+$: Minimum operating cost (per hour) for a thermal generator of type $t$.

$\text{per_mw_cost}_t \in \mathbb{R}^+$: Cost to generate one additional MW (per hour) for a thermal generator of type $t$.

$\text{startup_cost}_t \in \mathbb{R}^+$: Startup cost for thermal generator of type $t$.

$\text{hydro_load}_h \in \mathbb{R}^+$: Output for hydro generator $h$.

$\text{hydro_cost}_h \in \mathbb{R}^+$: Cost for operating hydro generator $h$.

$\text{hydro_startup_cost}_h \in \mathbb{R}^+$: Startup cost for hydro generator $h$.

$\text{hydro_height_reduction}_h \in \mathbb{R}^+$: Hourly reduction in reservoir height from operating hydro generator $h$.

Added:

$\text{max_power_incr}_t \in \mathbb{R}^+$: Maximum rate of change for power increase in thermal generator $MW$

$\text{max_power_decr}_t \in \mathbb{R}^+$: Maximum rate of change for power decrease in thermal generator $MW$

$\text{max_power_on}_t \in \mathbb{R}^+$: Limit power when units starts for thermal generator $MW$

$\text{max_power_off}_t \in \mathbb{R}^+$: Limit power when units stops for thermal generator $MW$

### Decision Variables

Changed:

$\text{ngen}_{t,i,p} \in [0,1]$: Indicates whether thermal generators $i$ of type $t$ is on in time period $p$.

$\text{output}_{t,i,p} \in \mathbb{R}^+$: Total power output from thermal generators $i$ of type $t$ in time period $p$.

$\text{nstart}_{t,i,p} \in [0,1]$: Indicates whether thermal generators $i$ of type $t$ starts in time period $p$.

Same as without differentiation:

$\text{hydro}_{h,p} \in [0,1]$: Indicates whether hydro generators $h$ is on in time period $p$.

$\text{hydro_start}_{h,p} \in [0,1]$: Indicates whether hydro generator $h$ starts in time period $p$.

$\text{height}_{p} \in \mathbb{R}^+$: Height of reservoir in time period $p$.

$\text{pumping}_{p} \in \mathbb{R}^+$: Power used to replenish reservoir in time period $p$.

Added:

$\text{nstop}_{t,i,p} \in [0,1]$: Indicates whether thermal generators $i$ of type $t$ stops in time period $p$.

### Objective Function

- **Cost**: Minimize the cost (in USD) to satisfy the predicted electricity demand.

\begin{equation}
\text{Minimize} \quad Z_{on} + Z_{extra} + Z_{startup} + Z_{hydro} + Z_{hydro\_startup}
\end{equation}

Changed: 

\begin{equation}
Z_{on} = \sum_{(t,p) \in \text{Types} \times \text{ThermalUnits}_t \times \text{Periods}}{\text{period_hours}_p*\text{base_cost}_t*\text{ngen}_{t,i,p}}
\end{equation}
\begin{equation}
Z_{extra} = \sum_{(t,i,p) \in \text{Types} \times \text{Periods}}{\text{period_hours}_p*\text{per_mw_cost}_t*(\sum_{ i \in \text{ThermalUnits}_t}\text{output}_{t,i,p} - \text{min_load}_t})
\end{equation}
\begin{equation}
Z_{startup} = \sum_{(t,i,p) \in \text{Types} \times \text{ThermalUnits}_t \times \text{Periods}}{\text{startup_cost}_t*\text{nstart}_{t,i,p}}
\end{equation}
\begin{equation}
Z_{hydro} = \sum_{(h,p) \in \text{HydroUnits} \times \text{Periods}}{\text{period_hours}_p*\text{hydro_cost}_h*\text{hydro}_{h,p}}
\end{equation}

Same as without differentiation:

\begin{equation}
Z_{hydro\_startup} = \sum_{(h,p) \in \text{HydroUnits} \times \text{Periods}}{\text{hydro_startup_cost}_h*\text{hydro_start}_{h,p}}
\end{equation}


### Constraints

Changed:

- **Demand**: Total power generated across all generator types must meet anticipated demand plus pumping for each time period $p$.

\begin{equation}
\sum_{(t,i) \in \text{Types} \times \text{ThermalUnits}_t}{\text{output}_{t,i,p}} +
\sum_{h \in \text{HydroUnits}}{\text{hydro_load}_h*\text{hydro}_{h,p}} \geq
\text{demand}_p + \text{pumping}_p \quad \forall p \in \text{Periods}
\end{equation}

- **Min/max generation**: Power generation must respect thermal generator min/max values.

\begin{equation}
\text{output}_{t,i,p} \geq \text{min_output}_t * \text{ngen}_{t,i,p} \quad \forall (t,i,p) \in \text{Types} \times \text{Periods} \times \text{ThermalUnits}
\end{equation}

\begin{equation}
\text{output}_{t,i,p} \leq \text{max_output}_t *\text{ngen}_{t,i,p} \quad \forall (t,i,p) \in \text{Types} \times \text{Periods}\times \text{ThermalUnits}
\end{equation}

- **Reserve**: Selected thermal generators plus hydro units must be able to satisfy demand that is as much as 15% above the prediction.

\begin{equation}
\sum_{(t,i) \in \text{Types} \times \text{ThermalUnits_t}}{\text{max_output}_t*\text{ngen}_{t,i,p}} +
\sum_{h \in \text{HydroUnits}}{\text{hydro_load}_h} \geq 1.15 * \text{demand}_p \quad \forall p \in \text{Periods}
\end{equation}

- **Thermal startup**: Establish relationship between thermal generator state and number of thermal startup (use $start0$ for period before the time horizon starts)

\begin{equation}
\text{ngen}_{t,i,p} \leq \text{ngen}_{t,i,p-1} + \text{nstart}_{t,i,p} \quad \forall (t,i,p) \in \text{Types} \times \text{ThermalUnits} \times \text{Periods}
\end{equation}

Same as without differentiation:

- **Hydro startup**: Establish relationship between hydro generator state and number of hydro startups (assume hydro plants are off at the beginning of the horizon)

\begin{equation}
\text{hydro}_{h,p} \leq \text{hydro}_{h,p-1} + \text{hydro_start}_{h,p} \quad \forall (h,p) \in \text{HydroUnits} \times \text{Periods}
\end{equation}

- **Reservoir height**: Track reservoir height.   Note that the height at the end of the final time period must equal the height at the beginning of the first.

- Reservoir level constraints: Height increases due to pumping activity and decreases due to hydroelectric generation.

\begin{equation}
\text{height}_{p} = \text{height}_{p-1}  + \text{period_hours}_{p}*\text{pumping}_{p}/3000 -
\sum_{h \in \text{HydroUnits}}{\text{period_hours}_{p}*\text{hydro_height_reduction}_{h}*\text{hydro}_{h,p}} \quad \forall p \in \text{Periods}
\end{equation}

- Cyclic constraint: Height at the first period must be equal to height at the last period.

\begin{equation}
\text{height}_{pfirst} = \text{height}_{plast}  + \text{period_hours}_{pfirst}*\text{pumping}_{pfirst}/3000 -
\sum_{h \in \text{HydroUnits}}{\text{period_hours}_{pfirst}*\text{hydro_height_reduction}_{h}*\text{hydro}_{h,pfirst}}
\end{equation}

Added:

-  Limit of the maximum rate of change when increasing during two periods and while turning on.

\begin{equation}
\text{output}_{t,i,p} - \text{output}_{t,i,p-1} \leq \text{max_power_incr}_t * \text{ngen}_{t,i,p} + (\text{max_power_on}_t - \text{max_power_incr}_t)*\text{nstart}_{t,i,p} \forall (t,i,p) \in \text{Types} \times \text{ThermalUnits} \times \text{Periods}
\end{equation}

-  Limit of the maximum rate of change when decreasing during two periods and while turning off.

\begin{equation}
\text{output}_{t,i,p-1} - \text{output}_{t,i,p} \leq \text{max_power_decr}_t * \text{ngen}_{t,i,p-1} + (\text{max_power_off}_t - \text{max_power_decr}_t)*\text{nstop}_{t,i,p} \forall (t,i,p) \in \text{Types} \times \text{ThermalUnits} \times \text{Periods}
\end{equation}

---
## Python Implementation

We import the Gurobi Python Module and other Python libraries.


In [1]:
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.0

## Input Data
We define all the input data of the model.

In [2]:
# Parameters

types = ["t0", "t1", "t2"]
units = [12, 10, 5]
nperiods = 5
maxstart0 = 5
hydrounits = 2

generators = [12, 10, 5]
period_hours = [6, 3, 6, 3, 6]
demand = [15000, 30000, 25000, 40000, 27000]
min_load = [850, 1250, 1500]
max_load = [2000, 1750, 4000]
base_cost = [1000, 2600, 3000]
per_mw_cost = [2, 1.3, 3]
startup_cost = [2000, 1000, 500]

hydro_load = [900, 1400]
hydro_cost = [90, 150]
hydro_height_reduction = [0.31, 0.47]
hydro_startup_cost = [1500, 1200]

#Added:
max_power_incr = [800,1200,1700]
max_power_decr = [800,1200,1700]
max_power_on = [1000,1500,2000]
max_power_off = [1000,1500,2000]

## Model Deployment

We create a **model** and the **variables**. For each time period, we have: a binary decision variable to capture if a thermal generators of a certain type is active or not (ngen), a binary variable to capture if a thermal generator of a certain type must be started (nstart), an other binary variable to designated if a thermal generator of a certain type must be turned off (nstop), a continuous decision variable to capture the total power output for each unit generator of each type (output), a binary decision variable that indicates whether a hydro unit is active (hydro), a binary decision variable that indicates whether a hydro unit must be started (hydrstart), a continuous decision variable that captures the amount of enery used to replenish the reservoir (pumping), and a continuous decision variable that captures the height of the reservoir (height).

In [3]:
model = gp.Model('PowerGeneration2')

#Changed
ngen, nstart, output, nstop = {}, {}, {}, {}

for i,name in enumerate(types):
    ngen[name] = model.addVars(units[i], nperiods, vtype=GRB.BINARY, name="thermal1")
    nstart[name] = model.addVars(units[i], nperiods, vtype=GRB.BINARY, name="nstart")
    nstop[name] = model.addVars(units[i], nperiods, vtype=GRB.BINARY, name="nstart")
    output[name] = model.addVars(units[i], nperiods, vtype=GRB.CONTINUOUS, name="genoutput", lb = 0.0 )
#ngen['t0'] is the variable ngen_{0,i,p}

hydro = model.addVars(hydrounits, nperiods, vtype=GRB.BINARY, name="hydro")
hydrostart = model.addVars(hydrounits, nperiods, vtype=GRB.BINARY, name="hydrostart")
pumping = model.addVars(nperiods, vtype=GRB.CONTINUOUS, name="pumping", lb = 0.0)
height = model.addVars(nperiods, vtype=GRB.CONTINUOUS, name="height", lb = 0.0)

Restricted license - for non-production use only - expires 2023-10-25


Next we insert the **constraints**:

Total power output for a thermal generator type depends on the number of generators of that type that are active.

In [4]:
# Respect minimum and maximum output per generator type

min_output = model.addConstrs( (output[name][unit,period] >= min_load[t] * ngen[name][unit,period]) 
                              for t,name in enumerate(types) for unit in range(units[t]) for period in range(nperiods) ) 

max_output = model.addConstrs( (output[name][unit,period] <= max_load[t] * ngen[name][unit,period]) 
                              for t,name in enumerate(types) for unit in range(units[t]) for period in range(nperiods) ) 

Total generator output (thermal plus hydro) for each time period must meet predicted demand plus pumping.

In [5]:
# Meet demand

meet_demand = model.addConstrs(gp.quicksum(output[name][unit,period] for t,name in enumerate(types) for unit in range(units[t])) 
                              + gp.quicksum(hydro_load[unit]*hydro[unit,period] for unit in range(hydrounits))
                               >= demand[period] 
                               + pumping[period]
                               for period in range(nperiods))

Maintain appropriate reservoir levels

In [6]:
# Reservoir levels

reservoir = model.addConstrs(height[period] == height[period-1] + period_hours[period]*pumping[period]/3000 -
                             gp.quicksum(hydro_height_reduction[unit]*period_hours[period]*hydro[unit,period] for unit in range(hydrounits))
                             for period in range(1,nperiods))

# cyclic - height at end must equal height at beginning
reservoir0 = model.addConstr(height[0] == height[nperiods-1] + period_hours[0]*pumping[0]/3000 -
                             gp.quicksum(hydro_height_reduction[unit]*period_hours[0]*hydro[unit,0]
                             for unit in range(hydrounits)))

Selected generators must be able to cope with an excess of demand.

In [7]:
# Provide sufficient reserve capacity

reserve = model.addConstrs(gp.quicksum(max_load[t]*ngen[name][unit, period] for t,name in enumerate(types) for unit in range(units[t])) >=
                           1.15*demand[period] - sum(hydro_load)
                           for period in range(nperiods))

Connect the decision variables that capture active thermal generators with the decision variables that count startups.

In [8]:
# Startup constraint

startup0 = model.addConstrs(gp.quicksum(ngen[name][unit,0] for unit in range(units[t])) <= maxstart0 +
                            gp.quicksum(nstart[name][unit,0] for unit in range(units[t])) for t,name in enumerate(types))


startup = model.addConstrs((ngen[name][unit,period] <= ngen[name][unit,period-1] + nstart[name][unit,period])
                           for t,name in enumerate(types) for unit in range(units[t]) for period in range(1,nperiods))

Connect hydro decision variables with hydro startup decision variables.

In [9]:
# Hydro startup constraint

hydro_startup0 = model.addConstrs((hydro[unit,0] <= hydrostart[unit,0])
                                    for unit in range(hydrounits))

hydro_startup = model.addConstrs((hydro[unit,period] <= hydro[unit,period-1] + hydrostart[unit,period])
                           for unit in range(hydrounits) for period in range(1,nperiods))

Limit of the maximum rate of change when increasing during two periods and while turning on.

In [10]:
limit_incr = model.addConstrs( output[name][unit, period] - output[name][unit,period-1] <= 
                              max_power_incr[t]*ngen[name][unit,period] + (max_power_on[t] - max_power_incr[t])*nstart[name][unit,period]
                              for t,name in enumerate(types) for unit in range(units[t]) for period in range(1,nperiods) )

limit_decr = model.addConstrs( output[name][unit, period-1] - output[name][unit,period] <= 
                              max_power_decr[t]*ngen[name][unit,period-1] + (max_power_off[t] - max_power_decr[t])*nstop[name][unit,period]
                              for t,name in enumerate(types) for unit in range(units[t]) for period in range(1,nperiods))

**Objective**: minimize total cost.  Cost consists of five components: the cost for running active thermal generation units, the cost to generate power beyond the minimum for each unit, the cost to start up thermal generation units, the cost to operate hydro units, and the cost to start up hydro units.

In [11]:
# Objective: minimize total cost

active = gp.quicksum(base_cost[t]*period_hours[period]*ngen[name][unit,period]
                    for t,name in enumerate(types) for unit in range(units[t]) for period in range(nperiods))

per_mw = gp.quicksum(per_mw_cost[t]*period_hours[period]*(gp.quicksum(output[name][unit,period] - min_load[t] for unit in range(units[t])))
                       for t,name in enumerate(types) for period in range(nperiods))

startup_obj = gp.quicksum(startup_cost[t]*nstart[name][unit,period]
                         for t,name in enumerate(types) for unit in range(units[t]) for period in range(nperiods))

hydro_active = gp.quicksum(hydro_cost[unit]*period_hours[period]*hydro[unit,period]
                           for unit in range(hydrounits) for period in range(nperiods))

hydro_startup = gp.quicksum(hydro_startup_cost[unit]*hydrostart[unit,period]
                            for unit in range(hydrounits) for period in range(nperiods))

model.setObjective(active + per_mw + startup_obj + hydro_active + hydro_startup)

Next, we start the optimization and Gurobi finds the optimal solution.

In [12]:
model.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 622 rows, 570 columns and 2120 nonzeros
Model fingerprint: 0x5d1492b0
Variable types: 145 continuous, 425 integer (425 binary)
Coefficient statistics:
  Matrix range     [1e-03, 4e+03]
  Objective range  [4e+00, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+00, 4e+04]
Found heuristic solution: objective 788064.00000
Presolve removed 3 rows and 162 columns
Presolve time: 0.00s
Presolved: 619 rows, 408 columns, 1978 nonzeros
Variable types: 145 continuous, 263 integer (261 binary)
Found heuristic solution: objective 720864.00000

Root relaxation: objective 1.979613e+05, 350 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 197961.330    0   67 720864.000 197961.3

---
## Analysis

Just considering the demand, the min/max output power: $\$150,900$

1) Considering the reserve constraint of 1.15 : $\$225,150$

2) Considering the startup cost and the constraint associated: $\$251,150$

3) Considering hydro plants: $\$207,400$

4) Considering the ramping: $\$221,360$

The anticipated demand for electricity over the 24-hour time window can be met for a total cost of $\$221,360$, as compared with the $\$1,000,630$ that was required to meet the same demand without differentiation. This shows that the precision of the model can changed the optimized cost by a factor 4.5. The detailed plan for each time period is as follows.

### Unit Commitments

The following shows the number of generators of each type that must be started in each time period to achieve this plan (recall that the model assumes that 5 per types are available at the beginning of the time horizon):

In [13]:
rows = ["Thermal" + str(t) for t,name in enumerate(types)]
Units = pd.DataFrame(columns=range(nperiods), index=rows, data=0.0)

for t,name in enumerate(types):
    for p in range(nperiods):
        Units.loc["Thermal"+str(t), p] = sum([ngen[name][unit,p].x for unit in range(units[t])])
Units

,0,1,2,3,4
Thermal0,8.0,11.0,12.0,12.0,12.0
Thermal1,0.0,6.0,2.0,9.0,3.0
Thermal2,0.0,0.0,0.0,1.0,0.0


In [14]:
rows = ["HydroA", "HydroB"]
hydrotable = pd.DataFrame(columns=range(nperiods), index=rows, data=0.0)

for p in range(nperiods):
    hydrotable.loc["HydroA", p] = int(hydro[0,p].x)
    hydrotable.loc["HydroB", p] = int(hydro[1,p].x)
hydrotable

,0,1,2,3,4
HydroA,0.0,1.0,1.0,1.0,0.0
HydroB,0.0,1.0,0.0,0.0,0.0


The hydro plants are more heavily used in the schedule than without the differentiation - the hydro plant are still both off at the first period.

The following shows the pumping that must be done in order to support this level of hydro activity

In [15]:
pumptable = pd.DataFrame(columns=range(nperiods), index=["Pumping"], data = 0.0)

for p in range(nperiods):
        pumptable.loc["Pumping", p] = pumping[p].x
pumptable

,0,1,2,3,4
Pumping,1000.0,0.0,1565.0,0.0,0.0


It is interesting to note that the plan simultaneously operates HydroA and pumps in time period 2. Also the level is completely recover at the period 4 while both hydro plant are off for the reserve factor 20%

While it might appear that costs could be reduced by operating the hydro plant at a lower load, in this model the hydro plants have fixed output.  If it makes economic sense to turn a hydro plant on, then we have to replenish the water drained, even if it means using some of the generated electricity to pump.

2.3.3) Using the same model but with an increasing of 5% regarding the reserve factor F: $\$245,135$

This is translated by an extra cost of $\$23,775$ compared with F = 15%

---

Let's look at the  optimal dual values of the LP relaxations

In [20]:
LP_relax = model.relax()

LP_relax.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 622 rows, 570 columns and 2120 nonzeros
Model fingerprint: 0x2602cff0
Coefficient statistics:
  Matrix range     [1e-03, 4e+03]
  Objective range  [4e+00, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+00, 5e+04]
Presolve removed 3 rows and 162 columns
Presolve time: 0.00s
Presolved: 619 rows, 408 columns, 1978 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.4196000e+06   6.670313e+03   0.000000e+00      0s
     317    2.2185419e+05   0.000000e+00   0.000000e+00      0s

Solved in 317 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.218541872e+05
